In [11]:
import PySimpleGUI as sg
import os.path
import requests
import cv2
import sqlite3
import os
import shutil
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
%matplotlib inline
from PIL import Image, ImageDraw, ImageFont

In [12]:
class Food_ai():
    headers = {'Prediction-Key': '93cf8213e5cb4a1e8432e0fece2449db',
           'Content-Type': 'application/octet-stream' }
    food_url = 'https://eastus.api.cognitive.microsoft.com/customvision/v3.0/Prediction/ea82d7ca-cbe9-40e2-b23d-2c3fe8daea71/detect/iterations/Iteration2/image'
    category_list = []
    def __init__(self,image_path):
        self.image_data = open(image_path, 'rb').read()
    
    def print_type_and_prob(self):
        response = requests.post(url=Food_ai.food_url, headers=Food_ai.headers, data=self.image_data)
#         print(eval(response.text))

        return eval(response.text)
#         return "Probability: " + str(eval(response.text)['predictions'][0]['probability']) + "\n" + "Type: " + eval(response.text)['predictions'][0]['tagName']
   # def return_category(self):
   #     return self.catego

In [13]:
class Take_picture():
    def __init__(self):
        cap = cv2.VideoCapture(0)#创建一个 VideoCapture 对象

        flag = 1 #设置一个标志，用来输出视频信息
        num = 1 #递增，用来保存文件名
        while(cap.isOpened()):#循环读取每一帧
            ret_flag, Vshow = cap.read() #返回两个参数，第一个是bool是否正常打开，第二个是照片数组，如果只设置一个则变成一个tumple包含bool和图片
            cv2.imshow("Take Food Picture",Vshow)  #窗口显示，显示名为 Capture_Test
            k = cv2.waitKey(1) & 0xFF #每帧数据延时 1ms，延时不能为 0，否则读取的结果会是静态帧
            if k == ord('s'):  #若检测到按键 ‘s’，打印字符串
                cv2.imwrite("C:/Users/Junhao's Alienware/food_recognition/food_picture/"+ str(num) + ".png", Vshow)
                print(cap.get(3)); #得到长宽
                print(cap.get(4));
                print("success to save"+str(num)+".png")
                print("-------------------------")
                num += 1
            elif k == ord('q'): #若检测到按键 ‘q’，退出
                break
        cap.release() #释放摄像头
        cv2.destroyAllWindows()#删除建立的全部窗口

In [14]:
class PictureResult:
    def __init__(self, image_path):
        self.img = Image.open(image_path)
        self.draw = ImageDraw.Draw(self.img)
        self.img_size = self.img.size
    def getRectangle(self, boundingBox):
        self.left = boundingBox['left'] * self.img_size[0]
        self.top = boundingBox['top'] * self.img_size[1]
        self.right = self.left + boundingBox['width'] * self.img_size[0]
        self.bottom = self.top + boundingBox['height'] * self.img_size[1]
#         return ((left, top), (right, bottom))
    def image_result(self, category):
        self.category = category
        self.font = ImageFont.truetype('arial',30)
        self.draw.rectangle(((self.left, self.top), (self.right, self.bottom)),outline='red')
        self.draw.text(((self.left+self.right)/2.4, (self.top+self.bottom)/2 ), self.category,(0,255,0), font=self.font)

In [15]:
# def getRectangle(faceDictionary):
#     rect = faceDictionary.face_rectangle
#     left = rect.left
#     top = rect.top
#     right = left + rect.width
#     bottom = top + rect.height
    
#     return ((left, top), (right, bottom))

In [16]:


# 连接到SQLite数据库
# 数据库文件是buy_value.db
# 如果文件不存在，会自动在当前目录创建:
conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
# 创建一个Cursor:
cursor = conn.cursor()
# 执行一条SQL语句，创建user表:
cursor.execute('create table customer (id varchar(20), value float)')
# 提交事务:
conn.commit()
# 关闭Connection:
conn.close()

sg.theme('Dark')   # 设置当前主题
file_list_column = [
    [
        sg.Text("功能按钮"),
#         sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
        sg.Button("拍照(s键为拍照，q键为退出)", key="Take_photo"),
#         '拍照（s是拍照，q是退出）', key=
        sg.Button("结算", key="Check_out"),
        sg.Button("重置", key="Reset")
        
#         '结算', key=
    ],
    [   
        sg.FolderBrowse('浏览结算'),
        sg.In(size=(40, 1), enable_events=True, key="-FOLDER-")
        
    ],
    [   
#         sg.Text("以下为食品照片清单：")
        sg.Text("以下为食品照片：")
    ],

    [
#         sg.Listbox(
#             values=[], enable_events=True, size=(40, 20), key="-FILE LIST-"
#         )
        sg.Image(key="-IMAGE-"),
    ],
]
image_viewer_column = [
#     [sg.Text("左侧为所有拍过的食品图片清单")],
    [sg.Text(size=(40, 1), key="-TOUT-")],
    [sg.Listbox(values=[], size=(40, 20), key="-POUT-")],
#     [sg.Text(size=(20, 1), key="-TOUT-")],
    [sg.Text(size=(40, 1), key="-SOUT-")],
#     [sg.Image(key="-IMAGE-")],
]
layout = [
    [
        sg.Column(file_list_column),
        sg.VSeperator(),
        sg.Column(image_viewer_column),
    ]
]
window = sg.Window("照片识别支付系统", layout)
while True:
    event, values = window.read()
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
    if event == "Reset":
        try:
            window["-TOUT-"].update('')
            window["-POUT-"].update('')
            window["-SOUT-"].update('')
            window["-IMAGE-"].update(filename=None)
            window["-FOLDER-"].update('')
#             window["-FILE LIST-"].update('')
            # 连接到SQLite数据库
            # 数据库文件是buy_value.db
            # 如果文件不存在，会自动在当前目录创建:
            conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
            # 创建一个Cursor:
            cursor = conn.cursor()
            # 执行一条SQL语句，创建user表:
            cursor.execute('create table customer (id varchar(20), value float)')
            # 提交事务:
            conn.commit()
            # 关闭Connection:
            conn.close()
        except:
            pass
    if event == "Take_photo":
        tke_picture = Take_picture()
    if event == "Check_out" or event == "-FOLDER-":
        if event == "Check_out":
            folder = "C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture"
            window["-FOLDER-"].update("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")
        if event == "-FOLDER-":
            folder = values["-FOLDER-"]
        
        try:
            file_list = os.listdir(folder)
            print(file_list)
        except:
            file_list = []
        try:
            fnames = [
                f
                for f in file_list
                if os.path.isfile(os.path.join(folder, f))
                and f.lower().endswith((".png", ".gif"))
            ]
    #         print(fnames)
#             window["-FILE LIST-"].update(fnames)
            print('update success')
            list1 = []
            for pic_name in fnames:
                print(pic_name)
    #             filename = os.path.join(
    #                 values["-FOLDER-"], pic_name
    #             )
                if event == "-FOLDER-":
                    filename = values["-FOLDER-"] + '\\' + pic_name
                if event == "Check_out":
                    filename = "C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture" + '\\' + pic_name

                image_process = PictureResult(filename)
    #            print(filename)
                food_rec = Food_ai(filename)
                response_text = food_rec.print_type_and_prob()
                for item in response_text['predictions']:
                    if (item['probability'] > 0.5):
                        list1.append(item['tagName'])
                        image_process.getRectangle(item['boundingBox'])
                        image_process.image_result(item['tagName'])
#                 print(food_rec.category_list)
# #                 list1.append(food_rec.category)
#                 list1 = food_rec.category_list
                image_final = image_process.img
                print(list1)
            image_final.save("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture\\checkout_picture.png")
            window["-IMAGE-"].update(filename="C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture\\checkout_picture.png")
            print(filename)
            #将扫描得出的产品类遍历输入buy_value数据库中
            for category_name in list1:           
                # 如果文件不存在，会自动在当前目录创建:
                conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/name_value.db")
                # 创建一个Cursor:
                cursor = conn.cursor()
                cursor.execute('select * from user where id=?', (category_name,))
                # 获得查询结果集:
                value = cursor.fetchall()
                print(value)
                # 提交事务:
                conn.commit()
                # 关闭Connection:
                conn.close()
                conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
                cursor = conn.cursor()
                cursor.execute("insert into customer (id, value) values (?, ?)", (value[0][0], float(value[0][1])))
                # 提交事务:
                conn.commit()
                # 关闭Connection:
                conn.close()
            #把数据输出到GUI
            conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
            cursor = conn.cursor()
            # 执行查询语句:
            cursor.execute('select * from customer')
            # 获得查询结果集:
            values = cursor.fetchall()
            # 提交事务:
            conn.commit()
            # 关闭Connection:
            conn.close()
            window["-TOUT-"].update('扫描产品及价格清单：')
            window["-POUT-"].update(values)
            conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
            cursor = conn.cursor()
            # 执行查询语句:
            cursor.execute('select sum(value) from customer')
            # 获得查询结果集:
            sum_value = cursor.fetchall()
            # 提交事务:
            conn.commit()
            # 关闭Connection:
            conn.close()
            window["-SOUT-"].update('总金额合计（元）：' + str(round(sum_value[0][0],2)))
            os.remove("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
            shutil.rmtree("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")
            os.mkdir("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")
    #     elif event == "-FILE LIST-":  
    #         try:
    #             filename = os.path.join(
    #                 values["-FOLDER-"], values["-FILE LIST-"][0]
    #             )
    # #             filename = values["-FILE LIST-"][0]
    #             print(filename)
    #             print(type(filename))
    #             window["-TOUT-"].update(filename)
    #             food_rec = Food_ai(filename)
    #             window["-POUT-"].update(food_rec.print_type_and_prob())
    #             window["-IMAGE-"].update(filename=filename)
    #             image_data = open(filename, 'rb').read()

    #         except:
    #             pass
        except:
            pass
try:
    os.remove("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
except:
    pass
try:
    os.mkdir("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")
except:
    pass
window.close()

['.ipynb_checkpoints', '1.png', 'Checkout_pic', 'Database', 'draw_IMG.ipynb', 'Face Analysis.ipynb', 'Food_picture', 'food_rec_ai.ipynb', 'food_rec_ai_multi.ipynb', 'Food_Reg_GUI.ipynb', 'Food_Reg_GUI_multi-revise1.ipynb', 'Food_Reg_GUI_multi.ipynb', 'Food_Reg_GUI_Single.ipynb', 'ice_cream1.jpeg', 'opencv_final.ipynb', 'opencv_test.ipynb', 'sqlite_database.ipynb', 'sqlite_list.ipynb', 'Untitled.ipynb', '捕获.PNG']
update success
1.png
[]
捕获.PNG
['Ice_cream', 'Jonnycake', 'Mango_pancake', 'Millet_congee']
C:/Users/Junhao's Alienware/food_recognition\捕获.PNG
[('Ice_cream', 27.1)]
[('Jonnycake', 18.4)]
[('Mango_pancake', 46.9)]
[('Millet_congee', 23.0)]
[]
update success
C:/Users/Junhao's Alienware/food_recognition\捕获.PNG


In [86]:
image_path = "C:/Users/Junhao's Alienware/Desktop\下载.png"

In [87]:
image_data = open(image_path, 'rb').read()

In [9]:
food_rec = Food_ai("C:/Users/Junhao's Alienware/Desktop/下载.png")

In [89]:
print(food_rec.print_type_and_prob())

{'probability': 0.99999666, 'tagId': '4d5086e5-535c-4f30-8e9f-eb2e90d2b37c', 'tagName': '冰激凌'}


In [17]:
os.remove("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")

PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: "C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db"

In [10]:
shutil.rmtree("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")

In [11]:
os.mkdir("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")